In [1]:
from os import listdir
from os.path import isfile, join
mypath = './dataset'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['batt_info.csv000',
 'devuse_4known_device.csv000',
 'mousewait.csv000.gz',
 'system_sysinfo_unique_normalized.csv000.gz',
 'plist_process_resource_util_hist.csv003.gz',
 'plist_process_resource_util_hist.csv002.gz',
 'plist_process_resource_util_hist.csv001.gz',
 'batt_acdc_events.csv000',
 'devuse_4known_device.csv001',
 'frgnd_v2.csv000',
 'frgnd_backgrnd_apps.csv000',
 'plist_process_resource_util_13wks.csv000.gz',
 'hw_metric_histo.csv001.gz',
 'hw_metric_histo.csv000.gz',
 'plist_process_resource_util_13wks.csv001.gz']

In [2]:
'frgnd_backgrnd_apps.csv000.gz',
'devuse_4known_device.csv000.gz',
'devuse_4known_device.csv001.gz',
'batt_acdc_events.csv000.gz',
'batt_info.csv000.gz',

('batt_info.csv000.gz',)

# EDA

## Loading data into DataFrames

In [3]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar 
from matplotlib import pyplot as plt
from dask.delayed import delayed

import pandas as pd
import os


list_of_fp=['frgnd_backgrnd_apps.csv000.gz',
'devuse_4known_device.csv000.gz',
'devuse_4known_device.csv001.gz',
'batt_acdc_events.csv000.gz',
'batt_info.csv000.gz']


#dfs = [delayed(pd.read_csv)(mypath+"/"+fn) for fn in list_of_fp]

#df = dd.from_delayed(dfs)

fp=mypath+"/"+'frgnd_backgrnd_apps.csv000'
fp2 = mypath+"/"+'devuse_4known_device.csv000'
fp3 = mypath+"/"+'devuse_4known_device.csv001'
fp4 = mypath+"/"+'batt_acdc_events.csv000'
fp5 = mypath+"/"+'batt_info.csv000'

frgnd_backgrnd_apps = dd.read_csv(fp,sep='')
devuse1 = dd.read_csv(fp2,sep='\t',error_bad_lines=False)
devuse2 = dd.read_csv(fp3,sep='\t',error_bad_lines=False,dtype={'duration': 'float64'})
batt_acdc = dd.read_csv(fp4,sep='\1')
batt_info = dd.read_csv(fp5,sep='\1',dtype={'designed_capacity': 'float64',
       'full_charge_capacity': 'float64'})



In [4]:
batt_info

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,battery_enum,chemistry,designed_capacity,full_charge_capacity,battery_count
npartitions=86,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,int64,object,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [5]:
batt_acdc

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,system_power_state,event_type,duration,battery_percent_remaining,battery_minutes_remaining
npartitions=536,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,int64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
#Inspecting the columns of a DataFrame
print(frgnd_backgrnd_apps.columns)
print(devuse1.columns)
print(devuse2.columns)
print(batt_acdc.columns)
print(batt_info.columns)

Index(['load_ts', 'batch_id', 'audit_zip', 'audit_internal_path', 'guid',
       'interval_start_utc', 'interval_end_utc', 'interval_local_start',
       'interval_local_end', 'event_dt', 'event_ts', 'event_name',
       'event_duration_ms', 'num_proc_samples', 'frgnd_proc_name',
       'frgnd_proc_dt', 'frgnd_proc_ts', 'frgnd_proc_duration_ms',
       'frgnd_proc_pkg', 'frgnd_captioned', 'frgnd_window_mode',
       'frgnd_non_responsive', 'proc_name', 'proc_cpu_utilization'],
      dtype='object')
Index(['load_ts', 'batch_id', 'audit_zip', 'audit_internal_path', 'guid',
       'interval_start_utc', 'interval_end_utc', 'interval_local_start',
       'interval_local_end', 'ts', 'dt', 'device', 'hw_name', 'name',
       'duration', 'status'],
      dtype='object')
Index(['load_ts', 'batch_id', 'audit_zip', 'audit_internal_path', 'guid',
       'interval_start_utc', 'interval_end_utc', 'interval_local_start',
       'interval_local_end', 'ts', 'dt', 'device', 'hw_name', 'name',
       'du

In [7]:
#Finding the common columns between the  DataFrames
from functools import reduce

columns = [set(frgnd_backgrnd_apps.columns),
    set(devuse1.columns),
    set(devuse2.columns),
    set(batt_acdc.columns),
          set(batt_info.columns)]
common_columns = list(reduce(lambda a, i: a.intersection(i), columns))
common_columns 

['interval_local_start',
 'audit_zip',
 'guid',
 'audit_internal_path',
 'batch_id',
 'load_ts',
 'interval_end_utc',
 'interval_local_end',
 'interval_start_utc']

Now that we have a set of common columns shared by all four of the DataFrames, let’s take a look at the first couple of rows of the frgnd_backgrnd_apps DataFrame.

In [8]:
frgnd_backgrnd_apps.head()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,event_dt,...,frgnd_proc_name,frgnd_proc_dt,frgnd_proc_ts,frgnd_proc_duration_ms,frgnd_proc_pkg,frgnd_captioned,frgnd_window_mode,frgnd_non_responsive,proc_name,proc_cpu_utilization
0,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,NaN,2020-10-09,2020-10-09 22:00:25.561,0,NaN,NaN,NaN,f,DSATray.exe,0.12
1,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,chrome.exe,2020-10-09,2020-10-09 22:28:43.737,802973,NaN,c1,W,f,GameScannerService.exe,0.39
2,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,chrome.exe,2020-10-09,2020-10-09 22:50:41.24,2343497,NaN,c1,W,f,GameScannerService.exe,0.85
3,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,NaN,2020-10-09,2020-10-09 20:14:48.094,0,NaN,NaN,NaN,f,MoUsoCoreWorker.exe,1.12
4,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,chrome.exe,2020-10-09,2020-10-09 22:50:41.24,2343497,NaN,c1,W,f,MsMpEng.exe,0.58


In [9]:
#Looking at the head of the frgnd_backgrnd_apps DataFrame
frgnd_backgrnd_apps[common_columns].head()

,interval_local_start,audit_zip,guid,audit_internal_path,batch_id,load_ts,interval_end_utc,interval_local_end,interval_start_utc
0,2020-10-09 20:14:10,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,003019b8c0ae454eb228609aa48ac8d1,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,20201011-200228,2020-10-12 03:07:56,2020-10-11 00:15:17,2020-10-10 20:15:17,2020-10-10 00:14:10
1,2020-10-09 20:14:10,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,003019b8c0ae454eb228609aa48ac8d1,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,20201011-200228,2020-10-12 03:07:56,2020-10-11 00:15:17,2020-10-10 20:15:17,2020-10-10 00:14:10
2,2020-10-09 20:14:10,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,003019b8c0ae454eb228609aa48ac8d1,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,20201011-200228,2020-10-12 03:07:56,2020-10-11 00:15:17,2020-10-10 20:15:17,2020-10-10 00:14:10
3,2020-10-09 20:14:10,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,003019b8c0ae454eb228609aa48ac8d1,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,20201011-200228,2020-10-12 03:07:56,2020-10-11 00:15:17,2020-10-10 20:15:17,2020-10-10 00:14:10
4,2020-10-09 20:14:10,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,003019b8c0ae454eb228609aa48ac8d1,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,20201011-200228,2020-10-12 03:07:56,2020-10-11 00:15:17,2020-10-10 20:15:17,2020-10-10 00:14:10


## Cleaning and transforming DataFrames

In [10]:
frgnd_backgrnd_apps_poi = frgnd_backgrnd_apps.columns.difference(common_columns)
frgnd_backgrnd_apps_poi

Index(['event_dt', 'event_duration_ms', 'event_name', 'event_ts',
       'frgnd_captioned', 'frgnd_non_responsive', 'frgnd_proc_dt',
       'frgnd_proc_duration_ms', 'frgnd_proc_name', 'frgnd_proc_pkg',
       'frgnd_proc_ts', 'frgnd_window_mode', 'num_proc_samples',
       'proc_cpu_utilization', 'proc_name'],
      dtype='object')

In [11]:
frgnd_backgrnd_apps[frgnd_backgrnd_apps_poi].head()

,event_dt,event_duration_ms,event_name,event_ts,frgnd_captioned,frgnd_non_responsive,frgnd_proc_dt,frgnd_proc_duration_ms,frgnd_proc_name,frgnd_proc_pkg,frgnd_proc_ts,frgnd_window_mode,num_proc_samples,proc_cpu_utilization,proc_name
0,2020-10-09,156317,DC_DISPLAY_OFF,2020-10-09 22:00:25.561,NaN,f,2020-10-09,0,NaN,NaN,2020-10-09 22:00:25.561,NaN,16,0.12,DSATray.exe
1,2020-10-09,5203164,DC_DISPLAY_ON,2020-10-09 22:03:01.879,c1,f,2020-10-09,802973,chrome.exe,NaN,2020-10-09 22:28:43.737,W,80,0.39,GameScannerService.exe
2,2020-10-09,5203164,DC_DISPLAY_ON,2020-10-09 22:03:01.879,c1,f,2020-10-09,2343497,chrome.exe,NaN,2020-10-09 22:50:41.24,W,234,0.85,GameScannerService.exe
3,2020-10-09,4773068,DC_DISPLAY_ON,2020-10-09 20:14:48.094,NaN,f,2020-10-09,0,NaN,NaN,2020-10-09 20:14:48.094,NaN,478,1.12,MoUsoCoreWorker.exe
4,2020-10-09,5203164,DC_DISPLAY_ON,2020-10-09 22:03:01.879,c1,f,2020-10-09,2343497,chrome.exe,NaN,2020-10-09 22:50:41.24,W,234,0.58,MsMpEng.exe


In [12]:
devuse1_poi = devuse1.columns.difference(common_columns)
devuse1_poi

Index(['device', 'dt', 'duration', 'hw_name', 'name', 'status', 'ts'], dtype='object')

In [13]:
devuse1[devuse1_poi].head()

,device,dt,duration,hw_name,name,status,ts
0,GUID_DEVINTERFACE_DISPLAY_ADAPTER,2019-11-10,16571,PCI\VEN_8086&DEV_0F31&SUBSYS_14DD1043&REV_0E,Intel(R) HD Graphics,4,2019-11-10 20:58:51
1,GUID_DEVINTERFACE_KEYBOARD,2019-11-11,82117,ACPI\VEN_ATK&DEV_3001,PC/AT Enhanced PS/2 Keyboard (101/102-Key),4,2019-11-11 21:00:33
2,GUID_DEVINTERFACE_MOUSE,2019-11-11,82117,ACPI\VEN_ETD&DEV_0108,ASUS Touchpad,4,2019-11-11 21:00:33
3,GUID_DEVICE_BATTERY,2019-11-11,82117,ACPI\VEN_PNP&DEV_0C0A,Microsoft ACPI-Compliant Control Method Battery,4,2019-11-11 21:00:33
4,GUID_DEVINTERFACE_HID,2019-11-11,82117,HID\VEN_ATK&DEV_4001,HID-compliant wireless radio controls,4,2019-11-11 21:00:33


In [14]:
devuse2_poi = devuse2.columns.difference(common_columns)
devuse2[devuse2_poi].head()

,device,dt,duration,hw_name,name,status,ts
0,GUID_DEVINTERFACE_DISK,2020-05-10,24953.0,SCSI\DiskSAMSUNG_MZNLN512HMJP-000MAV2,SAMSUNG MZNLN512HMJP-000L7,4,2020-05-10 22:48:53
1,GUID_DEVINTERFACE_VOLUME,2020-05-10,24953.0,STORAGE\Volume,Volume,4,2020-05-10 22:48:53
2,GUID_DEVINTERFACE_IMAGE,2020-05-10,24953.0,SUSD_VISTA_2008_WIN7_2008R2_64,Samsung Universal Scan Driver,4,2020-05-10 22:48:53
3,GUID_DEVINTERFACE_SENSOR,2020-05-10,24953.0,Sensors\SensorsSimulatorDriver,Microsoft Visual Studio Location Simulator Sensor,4,2020-05-10 22:48:53
4,GUID_DEVINTERFACE_USB_DEVICE,2020-05-10,24953.0,USB\VID_04F2&PID_B52C&REV_0029,USB Composite Device,4,2020-05-10 22:48:53


In [15]:
batt_acdc_poi = batt_acdc.columns.difference(common_columns)
batt_acdc[batt_acdc_poi].head()

,battery_minutes_remaining,battery_percent_remaining,dt,duration,event_type,system_power_state,ts
0,31.616667,75,2020-04-21,810.408,DC,POWERON,2020-04-21 09:33:26
1,73.766670,69,2020-04-21,13.573,DC,POWEROFF,2020-04-21 09:46:56
2,73.766670,69,2020-04-21,173.669,DC,POWERON,2020-04-21 09:47:10
3,503.700012,67,2020-04-21,38471.500,DC,POWERON,2020-04-21 09:50:03
4,0.000000,8,2020-04-21,43907.823,DC,POWEROFF,2020-04-21 20:31:15


In [16]:
batt_info_poi = batt_info.columns.difference(common_columns)
batt_info[batt_info_poi].head()

,battery_count,battery_enum,chemistry,designed_capacity,dt,full_charge_capacity,ts
0,1,1,LION,43092.0,2020-03-02,43320.0,2020-03-02 12:54:05
1,1,1,LION,43092.0,2020-03-02,43320.0,2020-03-02 12:57:12
2,1,-1,NaN,43092.0,2020-03-03,43320.0,2020-03-03 08:09:34
3,1,1,LION,43092.0,2020-03-03,43320.0,2020-03-03 13:06:26
4,1,-1,NaN,43092.0,2020-03-03,43320.0,2020-03-03 16:46:17


In order to help Dask read our dataset correctly, we’ll need to manually define a schema for our data instead of relying on type inference.

### Dealing with missing values

In [17]:
#Calculating the percentage of missing values by column
missing_values_frgnd_backgrnd = frgnd_backgrnd_apps.isnull().sum()

with ProgressBar():
    percent_missing_frgnd_backgrnd = ((missing_values_frgnd_backgrnd / frgnd_backgrnd_apps.index.size) * 100).compute()
percent_missing_frgnd_backgrnd

[########################################] | 100% Completed | 26.7s


load_ts                    0.000000
batch_id                   0.000000
audit_zip                  0.000000
audit_internal_path        0.000000
guid                       0.000000
interval_start_utc         0.000000
interval_end_utc           0.000000
interval_local_start       0.000000
interval_local_end         0.000000
event_dt                   0.000000
event_ts                   0.000000
event_name                 0.000000
event_duration_ms          0.000000
num_proc_samples           0.000000
frgnd_proc_name           11.777160
frgnd_proc_dt              0.000000
frgnd_proc_ts              0.000000
frgnd_proc_duration_ms     0.000000
frgnd_proc_pkg            98.686878
frgnd_captioned           12.805952
frgnd_window_mode         12.805952
frgnd_non_responsive       0.000000
proc_name                  0.000000
proc_cpu_utilization       0.000000
dtype: float64

In [18]:
missing_values_devuse1 = devuse1.isnull().sum()

with ProgressBar():
    percent_missing_devuse1 = ((missing_values_devuse1 / devuse1.index.size) * 100).compute()
percent_missing_devuse1

[###                                     ] | 8% Completed |  3min 31.7s

b'Skipping line 169363: expected 16 fields, saw 19\nSkipping line 169364: expected 16 fields, saw 19\nSkipping line 169434: expected 16 fields, saw 19\nSkipping line 169435: expected 16 fields, saw 19\nSkipping line 169504: expected 16 fields, saw 19\nSkipping line 169505: expected 16 fields, saw 19\nSkipping line 169579: expected 16 fields, saw 19\nSkipping line 169580: expected 16 fields, saw 19\nSkipping line 169630: expected 16 fields, saw 19\nSkipping line 169631: expected 16 fields, saw 19\nSkipping line 169701: expected 16 fields, saw 19\nSkipping line 169702: expected 16 fields, saw 19\nSkipping line 169771: expected 16 fields, saw 19\nSkipping line 169772: expected 16 fields, saw 19\nSkipping line 169845: expected 16 fields, saw 19\nSkipping line 169846: expected 16 fields, saw 19\nSkipping line 169914: expected 16 fields, saw 19\nSkipping line 169916: expected 16 fields, saw 19\nSkipping line 169965: expected 16 fields, saw 19\nSkipping line 169966: expected 16 fields, saw 19

[#######                                 ] | 19% Completed |  7min 47.7s

b'Skipping line 149688: expected 16 fields, saw 19\nSkipping line 149689: expected 16 fields, saw 19\nSkipping line 149754: expected 16 fields, saw 19\nSkipping line 149755: expected 16 fields, saw 19\nSkipping line 149797: expected 16 fields, saw 19\nSkipping line 149798: expected 16 fields, saw 19\nSkipping line 149839: expected 16 fields, saw 19\nSkipping line 149840: expected 16 fields, saw 19\nSkipping line 149903: expected 16 fields, saw 19\nSkipping line 149904: expected 16 fields, saw 19\nSkipping line 149966: expected 16 fields, saw 19\nSkipping line 149967: expected 16 fields, saw 19\nSkipping line 150030: expected 16 fields, saw 19\nSkipping line 150031: expected 16 fields, saw 19\nSkipping line 150093: expected 16 fields, saw 19\nSkipping line 150094: expected 16 fields, saw 19\nSkipping line 150157: expected 16 fields, saw 19\nSkipping line 150158: expected 16 fields, saw 19\nSkipping line 150199: expected 16 fields, saw 19\nSkipping line 150200: expected 16 fields, saw 19

[#############                           ] | 34% Completed | 14min  2.0s

b'Skipping line 9497: expected 16 fields, saw 19\nSkipping line 9498: expected 16 fields, saw 19\nSkipping line 9562: expected 16 fields, saw 19\nSkipping line 9563: expected 16 fields, saw 19\nSkipping line 9626: expected 16 fields, saw 19\nSkipping line 9627: expected 16 fields, saw 19\nSkipping line 9668: expected 16 fields, saw 19\nSkipping line 9669: expected 16 fields, saw 19\nSkipping line 9734: expected 16 fields, saw 19\nSkipping line 9735: expected 16 fields, saw 19\nSkipping line 9799: expected 16 fields, saw 19\nSkipping line 9800: expected 16 fields, saw 19\nSkipping line 9862: expected 16 fields, saw 19\nSkipping line 9863: expected 16 fields, saw 19\nSkipping line 9926: expected 16 fields, saw 19\nSkipping line 9927: expected 16 fields, saw 19\nSkipping line 9967: expected 16 fields, saw 19\nSkipping line 9968: expected 16 fields, saw 19\nSkipping line 10031: expected 16 fields, saw 19\nSkipping line 10032: expected 16 fields, saw 19\nSkipping line 10094: expected 16 fie

[########################                ] | 62% Completed | 25min 21.4s

b'Skipping line 101812: expected 16 fields, saw 17\n'


[########################################] | 100% Completed | 40min 37.3s


load_ts                 0.000000
batch_id                0.000000
audit_zip               0.000000
audit_internal_path     0.000000
guid                    0.000000
interval_start_utc      0.000000
interval_end_utc        0.000000
interval_local_start    0.000000
interval_local_end      0.000000
ts                      0.000000
dt                      0.000000
device                  0.000009
hw_name                 0.002434
name                    0.405555
duration                0.000000
status                  0.000000
dtype: float64

In [19]:
missing_values_devuse2 = devuse2.isnull().sum()

with ProgressBar():
    percent_missing_devuse2 = ((missing_values_devuse2 / devuse2.index.size) * 100).compute()
percent_missing_devuse2

[###############                         ] | 38% Completed |  8min 39.1s

b'Skipping line 88364: expected 16 fields, saw 17\n'


[#################                       ] | 43% Completed |  9min 45.3s

b'Skipping line 37400: expected 16 fields, saw 19\nSkipping line 37401: expected 16 fields, saw 19\nSkipping line 37453: expected 16 fields, saw 19\nSkipping line 37454: expected 16 fields, saw 19\nSkipping line 37503: expected 16 fields, saw 19\nSkipping line 37504: expected 16 fields, saw 19\nSkipping line 37598: expected 16 fields, saw 19\nSkipping line 37599: expected 16 fields, saw 19\nSkipping line 37684: expected 16 fields, saw 19\nSkipping line 37686: expected 16 fields, saw 19\nSkipping line 37723: expected 16 fields, saw 19\nSkipping line 37725: expected 16 fields, saw 19\nSkipping line 37759: expected 16 fields, saw 19\nSkipping line 37760: expected 16 fields, saw 19\nSkipping line 37798: expected 16 fields, saw 19\nSkipping line 37800: expected 16 fields, saw 19\nSkipping line 37881: expected 16 fields, saw 19\nSkipping line 37882: expected 16 fields, saw 19\nSkipping line 37911: expected 16 fields, saw 19\nSkipping line 37912: expected 16 fields, saw 19\nSkipping line 3793

[#################                       ] | 43% Completed |  9min 46.1s

b'Skipping line 81548: expected 16 fields, saw 19\nSkipping line 81549: expected 16 fields, saw 19\nSkipping line 81583: expected 16 fields, saw 19\nSkipping line 81584: expected 16 fields, saw 19\nSkipping line 81677: expected 16 fields, saw 19\nSkipping line 81678: expected 16 fields, saw 19\nSkipping line 81708: expected 16 fields, saw 19\nSkipping line 81709: expected 16 fields, saw 19\nSkipping line 81816: expected 16 fields, saw 19\nSkipping line 81817: expected 16 fields, saw 19\nSkipping line 81920: expected 16 fields, saw 19\nSkipping line 81921: expected 16 fields, saw 19\nSkipping line 81978: expected 16 fields, saw 19\nSkipping line 81979: expected 16 fields, saw 19\nSkipping line 82010: expected 16 fields, saw 19\nSkipping line 82011: expected 16 fields, saw 19\nSkipping line 82104: expected 16 fields, saw 19\nSkipping line 82105: expected 16 fields, saw 19\nSkipping line 82140: expected 16 fields, saw 19\nSkipping line 82141: expected 16 fields, saw 19\nSkipping line 8220

[#################                       ] | 43% Completed |  9min 46.9s

b'Skipping line 123398: expected 16 fields, saw 19\nSkipping line 123399: expected 16 fields, saw 19\nSkipping line 123454: expected 16 fields, saw 19\nSkipping line 123455: expected 16 fields, saw 19\nSkipping line 123546: expected 16 fields, saw 19\nSkipping line 123547: expected 16 fields, saw 19\nSkipping line 123598: expected 16 fields, saw 19\nSkipping line 123599: expected 16 fields, saw 19\nSkipping line 123670: expected 16 fields, saw 19\nSkipping line 123671: expected 16 fields, saw 19\nSkipping line 123721: expected 16 fields, saw 19\nSkipping line 123723: expected 16 fields, saw 19\nSkipping line 123749: expected 16 fields, saw 19\nSkipping line 123750: expected 16 fields, saw 19\nSkipping line 123826: expected 16 fields, saw 19\nSkipping line 123827: expected 16 fields, saw 19\nSkipping line 123874: expected 16 fields, saw 19\nSkipping line 123875: expected 16 fields, saw 19\nSkipping line 123922: expected 16 fields, saw 19\nSkipping line 123924: expected 16 fields, saw 19

[########################################] | 100% Completed | 22min  5.6s


load_ts                 0.000000e+00
batch_id                0.000000e+00
audit_zip               0.000000e+00
audit_internal_path     0.000000e+00
guid                    0.000000e+00
interval_start_utc      0.000000e+00
interval_end_utc        0.000000e+00
interval_local_start    0.000000e+00
interval_local_end      0.000000e+00
ts                      0.000000e+00
dt                      0.000000e+00
device                  2.430506e-05
hw_name                 2.242839e-03
name                    4.210620e-01
duration                3.984435e-07
status                  0.000000e+00
dtype: float64

In [20]:
missing_values_batt_acdc = batt_acdc.isnull().sum()

with ProgressBar():
    percent_missing_batt_acdc = ((missing_values_batt_acdc / batt_acdc.index.size) * 100).compute()
percent_missing_batt_acdc

[########################################] | 100% Completed | 10min  3.7s


load_ts                      0.0
batch_id                     0.0
audit_zip                    0.0
audit_internal_path          0.0
guid                         0.0
interval_start_utc           0.0
interval_end_utc             0.0
interval_local_start         0.0
interval_local_end           0.0
ts                           0.0
dt                           0.0
system_power_state           0.0
event_type                   0.0
duration                     0.0
battery_percent_remaining    0.0
battery_minutes_remaining    0.0
dtype: float64

In [21]:
missing_values_batt_info = batt_acdc.isnull().sum()

with ProgressBar():
    percent_missing_batt_info = ((missing_values_batt_info / batt_info.index.size) * 100).compute()
percent_missing_batt_info

[########################################] | 100% Completed | 11min 50.6s


load_ts                      0.0
batch_id                     0.0
audit_zip                    0.0
audit_internal_path          0.0
guid                         0.0
interval_start_utc           0.0
interval_end_utc             0.0
interval_local_start         0.0
interval_local_end           0.0
ts                           0.0
dt                           0.0
system_power_state           0.0
event_type                   0.0
duration                     0.0
battery_percent_remaining    0.0
battery_minutes_remaining    0.0
dtype: float64

###  DROPPING COLUMNS WITH MISSING VALUES

Now that we know what we have to work with, we’ll start by dropping any columns that most of their values missing.
We don’t likely stand to gain much information by keeping such a sparse column around, so we’ll remove it from our dataset.

In [22]:
#Dropping columns that have more than 90% missing values
columns_to_drop_frgnd_backgrnd = list(percent_missing_frgnd_backgrnd[percent_missing_frgnd_backgrnd >= 90].index)
frgnd_backgrnd_clean_stage1 = frgnd_backgrnd_apps.drop(columns_to_drop_frgnd_backgrnd, axis=1)

In [23]:
columns_to_drop_frgnd_backgrnd

['frgnd_proc_pkg']

In [24]:
#Dropping columns that have more than 90% missing values
columns_to_drop_devuse1 = list(percent_missing_devuse1[percent_missing_devuse1 >= 90].index)
devuse1_clean_stage1 = devuse1.drop(columns_to_drop_devuse1, axis=1)

In [25]:
columns_to_drop_devuse1

[]

In [26]:
#Dropping columns that have more than 90% missing values
columns_to_drop_devuse2 = list(percent_missing_devuse2[percent_missing_devuse2 >= 90].index)
devuse2_clean_stage1 = devuse2.drop(columns_to_drop_devuse2, axis=1)

In [27]:
columns_to_drop_devuse2

[]

In [28]:
#Dropping columns that have more than 90% missing values
columns_to_drop_batt_acdc = list(percent_missing_batt_acdc[percent_missing_batt_acdc>= 90].index)
batt_acdc_clean_stage1 = batt_acdc.drop(columns_to_drop_batt_acdc, axis=1)

In [29]:
columns_to_drop_batt_acdc

[]

In [30]:
#Dropping columns that have more than 90% missing values
columns_to_drop_batt_info = list(percent_missing_batt_info[percent_missing_batt_info >= 90].index)
batt_info_clean_stage1 = batt_info.drop(columns_to_drop_batt_info, axis=1)

In [31]:
columns_to_drop_batt_info

[]

## DROPPING ROWS WITH MISSING DATA

In [32]:
#Dropping rows with missing data
frgnd_backgrnd_rows_to_drop = list(percent_missing_frgnd_backgrnd[(percent_missing_frgnd_backgrnd > 0) & (percent_missing_frgnd_backgrnd < 5)].index)
frgnd_backgrnd_clean_stage2 = frgnd_backgrnd_clean_stage1.dropna(subset=frgnd_backgrnd_rows_to_drop)

In [33]:
frgnd_backgrnd_rows_to_drop

[]

In [34]:
#Dropping rows with missing data
devuse1_rows_to_drop = list(percent_missing_devuse1[(percent_missing_devuse1 > 0) & (percent_missing_devuse1 < 5)].index)
devuse1_clean_stage2 = devuse1_clean_stage1.dropna(subset=devuse1_rows_to_drop)

In [35]:
devuse1_rows_to_drop

['device', 'hw_name', 'name']

In [36]:
#Dropping rows with missing data
devuse2_rows_to_drop = list(percent_missing_devuse1[(percent_missing_devuse2 > 0) & (percent_missing_devuse2 < 5)].index)
devuse2_clean_stage2 = devuse2_clean_stage1.dropna(subset=devuse2_rows_to_drop)

In [37]:
devuse2_rows_to_drop

['device', 'hw_name', 'name', 'duration']

In [38]:
#Finding the datatypes of the remaining columns that contain missing value
remaining_columns_to_clean = list(percent_missing_frgnd_backgrnd[(percent_missing_frgnd_backgrnd >= 5)].index)
#frgnd_backgrnd_clean_stage2.dtypes[remaining_columns_to_clean]

In [39]:
remaining_columns_to_clean

['frgnd_proc_name', 'frgnd_proc_pkg', 'frgnd_captioned', 'frgnd_window_mode']

### Parse Date

In [40]:
frgnd_backgrnd_apps_poi=frgnd_backgrnd_clean_stage2.columns.difference(common_columns)
frgnd_backgrnd_apps_poi

Index(['event_dt', 'event_duration_ms', 'event_name', 'event_ts',
       'frgnd_captioned', 'frgnd_non_responsive', 'frgnd_proc_dt',
       'frgnd_proc_duration_ms', 'frgnd_proc_name', 'frgnd_proc_ts',
       'frgnd_window_mode', 'num_proc_samples', 'proc_cpu_utilization',
       'proc_name'],
      dtype='object')

In [41]:
frgnd_backgrnd_clean_stage2[frgnd_backgrnd_apps_poi].head()

,event_dt,event_duration_ms,event_name,event_ts,frgnd_captioned,frgnd_non_responsive,frgnd_proc_dt,frgnd_proc_duration_ms,frgnd_proc_name,frgnd_proc_ts,frgnd_window_mode,num_proc_samples,proc_cpu_utilization,proc_name
0,2020-10-09,156317,DC_DISPLAY_OFF,2020-10-09 22:00:25.561,NaN,f,2020-10-09,0,NaN,2020-10-09 22:00:25.561,NaN,16,0.12,DSATray.exe
1,2020-10-09,5203164,DC_DISPLAY_ON,2020-10-09 22:03:01.879,c1,f,2020-10-09,802973,chrome.exe,2020-10-09 22:28:43.737,W,80,0.39,GameScannerService.exe
2,2020-10-09,5203164,DC_DISPLAY_ON,2020-10-09 22:03:01.879,c1,f,2020-10-09,2343497,chrome.exe,2020-10-09 22:50:41.24,W,234,0.85,GameScannerService.exe
3,2020-10-09,4773068,DC_DISPLAY_ON,2020-10-09 20:14:48.094,NaN,f,2020-10-09,0,NaN,2020-10-09 20:14:48.094,NaN,478,1.12,MoUsoCoreWorker.exe
4,2020-10-09,5203164,DC_DISPLAY_ON,2020-10-09 22:03:01.879,c1,f,2020-10-09,2343497,chrome.exe,2020-10-09 22:50:41.24,W,234,0.58,MsMpEng.exe


In [42]:
with ProgressBar():
    frgnd_backgrnd_event_dt_value_count = frgnd_backgrnd_clean_stage2['event_dt'].value_counts().compute()

[########################################] | 100% Completed | 27.3s


In [43]:
frgnd_backgrnd_event_dt_value_count

2020-10-22    70024
2020-10-14    69961
2020-10-15    69882
2020-10-21    69882
2020-10-19    69163
              ...  
2021-01-02    12338
2021-01-01    11075
2021-01-07     6497
2020-10-09      881
2021-01-08       10
Name: event_dt, Length: 92, dtype: int64

In [44]:
frgnd_backgrnd_clean_stage2.head()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,event_dt,...,num_proc_samples,frgnd_proc_name,frgnd_proc_dt,frgnd_proc_ts,frgnd_proc_duration_ms,frgnd_captioned,frgnd_window_mode,frgnd_non_responsive,proc_name,proc_cpu_utilization
0,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,16,NaN,2020-10-09,2020-10-09 22:00:25.561,0,NaN,NaN,f,DSATray.exe,0.12
1,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,80,chrome.exe,2020-10-09,2020-10-09 22:28:43.737,802973,c1,W,f,GameScannerService.exe,0.39
2,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,234,chrome.exe,2020-10-09,2020-10-09 22:50:41.24,2343497,c1,W,f,GameScannerService.exe,0.85
3,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,478,NaN,2020-10-09,2020-10-09 20:14:48.094,0,NaN,NaN,f,MoUsoCoreWorker.exe,1.12
4,2020-10-12 03:07:56,20201011-200228,2020101115-i-040171543e1bfef37-ud0Qz3DtGaeMoEs...,V8_1_FRGND_BACKGRND_APPS20201010201454.V8,003019b8c0ae454eb228609aa48ac8d1,2020-10-10 00:14:10,2020-10-11 00:15:17,2020-10-09 20:14:10,2020-10-10 20:15:17,2020-10-09,...,234,chrome.exe,2020-10-09,2020-10-09 22:50:41.24,2343497,c1,W,f,MsMpEng.exe,0.58


In [45]:
with ProgressBar():
    frgnd_backgrnd_guid_value_count = frgnd_backgrnd_clean_stage2['guid'].value_counts().compute()

[########################################] | 100% Completed | 26.7s


In [46]:
frgnd_backgrnd_guid_value_count

0af707e7d51e4fb6812a596de9fef679    34419
9c4e19608ece40f5b1db5b37d0091a2f    31844
007729cb039e4e1aaa6c4a86c8c436e5    30205
8189859393774a879a54ce8aaeebc181    28970
c69315235fa14225b0cfbc6dbb999e4e    27946
                                    ...  
db970d2a18d94ceb9f1ef1da8b178577       10
dcb2d536665a46daaa3d2151a546bf4d       10
de941903d7474f33bf9243b40cad4766       10
c172bd39a6884990a227743bd57e54de       10
3c07024d3df040e29d72d2f25a86e96e        9
Name: guid, Length: 2083, dtype: int64

There are 2083 unique identitfier in frgnd_backgrnd

In [47]:
devuse1_clean_stage2.head()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,device,hw_name,name,duration,status
0,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-10 20:58:51,2019-11-10,GUID_DEVINTERFACE_DISPLAY_ADAPTER,PCI\VEN_8086&DEV_0F31&SUBSYS_14DD1043&REV_0E,Intel(R) HD Graphics,16571,4
1,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVINTERFACE_KEYBOARD,ACPI\VEN_ATK&DEV_3001,PC/AT Enhanced PS/2 Keyboard (101/102-Key),82117,4
2,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVINTERFACE_MOUSE,ACPI\VEN_ETD&DEV_0108,ASUS Touchpad,82117,4
3,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVICE_BATTERY,ACPI\VEN_PNP&DEV_0C0A,Microsoft ACPI-Compliant Control Method Battery,82117,4
4,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVINTERFACE_HID,HID\VEN_ATK&DEV_4001,HID-compliant wireless radio controls,82117,4


In [48]:
with ProgressBar():
    device1_guid_value_count = devuse1_clean_stage2['guid'].value_counts().compute()

[######                                  ] | 16% Completed |  6min 40.5s

b'Skipping line 169363: expected 16 fields, saw 19\nSkipping line 169364: expected 16 fields, saw 19\nSkipping line 169434: expected 16 fields, saw 19\nSkipping line 169435: expected 16 fields, saw 19\nSkipping line 169504: expected 16 fields, saw 19\nSkipping line 169505: expected 16 fields, saw 19\nSkipping line 169579: expected 16 fields, saw 19\nSkipping line 169580: expected 16 fields, saw 19\nSkipping line 169630: expected 16 fields, saw 19\nSkipping line 169631: expected 16 fields, saw 19\nSkipping line 169701: expected 16 fields, saw 19\nSkipping line 169702: expected 16 fields, saw 19\nSkipping line 169771: expected 16 fields, saw 19\nSkipping line 169772: expected 16 fields, saw 19\nSkipping line 169845: expected 16 fields, saw 19\nSkipping line 169846: expected 16 fields, saw 19\nSkipping line 169914: expected 16 fields, saw 19\nSkipping line 169916: expected 16 fields, saw 19\nSkipping line 169965: expected 16 fields, saw 19\nSkipping line 169966: expected 16 fields, saw 19

[###############                         ] | 39% Completed | 16min 16.9s

b'Skipping line 101812: expected 16 fields, saw 17\n'


[##################################      ] | 85% Completed | 35min  9.5s

b'Skipping line 9497: expected 16 fields, saw 19\nSkipping line 9498: expected 16 fields, saw 19\nSkipping line 9562: expected 16 fields, saw 19\nSkipping line 9563: expected 16 fields, saw 19\nSkipping line 9626: expected 16 fields, saw 19\nSkipping line 9627: expected 16 fields, saw 19\nSkipping line 9668: expected 16 fields, saw 19\nSkipping line 9669: expected 16 fields, saw 19\nSkipping line 9734: expected 16 fields, saw 19\nSkipping line 9735: expected 16 fields, saw 19\nSkipping line 9799: expected 16 fields, saw 19\nSkipping line 9800: expected 16 fields, saw 19\nSkipping line 9862: expected 16 fields, saw 19\nSkipping line 9863: expected 16 fields, saw 19\nSkipping line 9926: expected 16 fields, saw 19\nSkipping line 9927: expected 16 fields, saw 19\nSkipping line 9967: expected 16 fields, saw 19\nSkipping line 9968: expected 16 fields, saw 19\nSkipping line 10031: expected 16 fields, saw 19\nSkipping line 10032: expected 16 fields, saw 19\nSkipping line 10094: expected 16 fie

[##################################      ] | 85% Completed | 35min 16.0s

b'Skipping line 149688: expected 16 fields, saw 19\nSkipping line 149689: expected 16 fields, saw 19\nSkipping line 149754: expected 16 fields, saw 19\nSkipping line 149755: expected 16 fields, saw 19\nSkipping line 149797: expected 16 fields, saw 19\nSkipping line 149798: expected 16 fields, saw 19\nSkipping line 149839: expected 16 fields, saw 19\nSkipping line 149840: expected 16 fields, saw 19\nSkipping line 149903: expected 16 fields, saw 19\nSkipping line 149904: expected 16 fields, saw 19\nSkipping line 149966: expected 16 fields, saw 19\nSkipping line 149967: expected 16 fields, saw 19\nSkipping line 150030: expected 16 fields, saw 19\nSkipping line 150031: expected 16 fields, saw 19\nSkipping line 150093: expected 16 fields, saw 19\nSkipping line 150094: expected 16 fields, saw 19\nSkipping line 150157: expected 16 fields, saw 19\nSkipping line 150158: expected 16 fields, saw 19\nSkipping line 150199: expected 16 fields, saw 19\nSkipping line 150200: expected 16 fields, saw 19

[########################################] | 100% Completed | 40min 49.4s


In [49]:
device1_guid_value_count

aac4a19cfed947489e7325756559a222    62531
a16fc9dae5e54c388f47d2ba559b98ef    38673
0161e19863ee4d59a4f6ed25420bd3b8    35927
0194bebaef2c458daf29f64271e440db    34160
00f8207451df4a65b7b711efbbd5dae4    34120
                                    ...  
043579b1347f4a159e89c2740a76a75f       59
001a032567ef4f9eb7a9d9757a025ee1       49
b6d98b7fc7f74c568a4d99001bbc41ea       48
7682073d644348269f6ba3e3d63d241d       31
7037a210aebc4bb7ac91294793dfcf74        1
Name: guid, Length: 64843, dtype: int64

In [50]:
with ProgressBar():
    device1_guid_value_count = devuse1_clean_stage2['guid'].value_counts().compute()

[######                                  ] | 16% Completed |  6min 38.5s

b'Skipping line 169363: expected 16 fields, saw 19\nSkipping line 169364: expected 16 fields, saw 19\nSkipping line 169434: expected 16 fields, saw 19\nSkipping line 169435: expected 16 fields, saw 19\nSkipping line 169504: expected 16 fields, saw 19\nSkipping line 169505: expected 16 fields, saw 19\nSkipping line 169579: expected 16 fields, saw 19\nSkipping line 169580: expected 16 fields, saw 19\nSkipping line 169630: expected 16 fields, saw 19\nSkipping line 169631: expected 16 fields, saw 19\nSkipping line 169701: expected 16 fields, saw 19\nSkipping line 169702: expected 16 fields, saw 19\nSkipping line 169771: expected 16 fields, saw 19\nSkipping line 169772: expected 16 fields, saw 19\nSkipping line 169845: expected 16 fields, saw 19\nSkipping line 169846: expected 16 fields, saw 19\nSkipping line 169914: expected 16 fields, saw 19\nSkipping line 169916: expected 16 fields, saw 19\nSkipping line 169965: expected 16 fields, saw 19\nSkipping line 169966: expected 16 fields, saw 19

[###############                         ] | 39% Completed | 16min  8.6s

b'Skipping line 101812: expected 16 fields, saw 17\n'


[##################################      ] | 85% Completed | 35min  6.0s

b'Skipping line 9497: expected 16 fields, saw 19\nSkipping line 9498: expected 16 fields, saw 19\nSkipping line 9562: expected 16 fields, saw 19\nSkipping line 9563: expected 16 fields, saw 19\nSkipping line 9626: expected 16 fields, saw 19\nSkipping line 9627: expected 16 fields, saw 19\nSkipping line 9668: expected 16 fields, saw 19\nSkipping line 9669: expected 16 fields, saw 19\nSkipping line 9734: expected 16 fields, saw 19\nSkipping line 9735: expected 16 fields, saw 19\nSkipping line 9799: expected 16 fields, saw 19\nSkipping line 9800: expected 16 fields, saw 19\nSkipping line 9862: expected 16 fields, saw 19\nSkipping line 9863: expected 16 fields, saw 19\nSkipping line 9926: expected 16 fields, saw 19\nSkipping line 9927: expected 16 fields, saw 19\nSkipping line 9967: expected 16 fields, saw 19\nSkipping line 9968: expected 16 fields, saw 19\nSkipping line 10031: expected 16 fields, saw 19\nSkipping line 10032: expected 16 fields, saw 19\nSkipping line 10094: expected 16 fie

[##################################      ] | 85% Completed | 35min 10.3s

b'Skipping line 149688: expected 16 fields, saw 19\nSkipping line 149689: expected 16 fields, saw 19\nSkipping line 149754: expected 16 fields, saw 19\nSkipping line 149755: expected 16 fields, saw 19\nSkipping line 149797: expected 16 fields, saw 19\nSkipping line 149798: expected 16 fields, saw 19\nSkipping line 149839: expected 16 fields, saw 19\nSkipping line 149840: expected 16 fields, saw 19\nSkipping line 149903: expected 16 fields, saw 19\nSkipping line 149904: expected 16 fields, saw 19\nSkipping line 149966: expected 16 fields, saw 19\nSkipping line 149967: expected 16 fields, saw 19\nSkipping line 150030: expected 16 fields, saw 19\nSkipping line 150031: expected 16 fields, saw 19\nSkipping line 150093: expected 16 fields, saw 19\nSkipping line 150094: expected 16 fields, saw 19\nSkipping line 150157: expected 16 fields, saw 19\nSkipping line 150158: expected 16 fields, saw 19\nSkipping line 150199: expected 16 fields, saw 19\nSkipping line 150200: expected 16 fields, saw 19

[########################################] | 100% Completed | 40min 47.2s
